# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
N. Wang  ->  N. Wang  |  ['N. Wang']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Z.-L. Xie  ->  Z.-L. Xie  |  ['Z.-L. Xie']
J. Wolf  ->  D. J. Wolf  |  ['J. Wolf']
F. Walter  ->  F. Walter  |  ['F. Walter']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
J. Li  ->  J. Li  |  ['J. Li']
C. Fendt  ->  C. Fendt  |  ['C. Fendt']


Arxiv has 61 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.15893
extracting tarball to tmp_2505.15893...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.15896


extracting tarball to tmp_2505.15896... done.
Retrieving document from  https://arxiv.org/e-print/2505.15923
extracting tarball to tmp_2505.15923...

 done.


...

extracting tarball to tmp_2505.16562... done.
Retrieving document from  https://arxiv.org/e-print/2505.16846


extracting tarball to tmp_2505.16846...

 done.


C. Fendt  ->  C. Fendt  |  ['C. Fendt']


Found 75 bibliographic references in tmp_2505.16846/mainoutflow.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.15923-b31b1b.svg)](https://arxiv.org/abs/2505.15923) | **Discovery and characterization of 25 new quasars at 4.6 < z < 6.9 from wide-field multi-band surveys**  |
|| <mark>S. Belladitta</mark>, et al. -- incl., <mark>E. Bañados</mark>, <mark>Z.-L. Xie</mark>, <mark>J. Wolf</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-05-23*|
|*Comments*| *25 pages, 8 figures, 12 tables, Accepted for publication in A&A*|
|**Abstract**|            Luminous quasars at $z>4$ provide key insights into the early Universe. Their rarity necessitates wide-field multi-band surveys to efficiently separate them from the main astrophysical contaminants (i.e., ultracool dwarfs). To expand the sample of high-$z$ quasars, we conducted targeted selections using optical, infrared, and radio surveys, complemented by literature-based quasar candidate catalogs. In this paper, we report the discovery of \nqsos\ new quasars at $4.6<z<6.9$ (six at $z\geq6.5$), with $M_{1450}$ between $-$25.4 and $-$27.0. We also present new spectra of six $z>6.5$ quasars we selected, but whose independent discovery has already been published in the literature. Three of the newly discovered quasars are strong radio emitters (L$_{1.4~\rm GHz}$$=0.09-1.0\times$10$^{34}$erg s$^{-1}$ Hz$^{-1}$). Among them, one source at $z=4.71$ exhibits typical blazar-like properties, including a flat radio spectrum, radio-loudness $\sim$1000, and multi-frequency variability. It is also detected by SRG/eROSITA X-ray telescope (f$_{\rm 0.2-2.3keV} \sim 1.3\times10^{-13}$erg s$^{-1}$ cm$^{-2}$). In addition, for seven $6.3<z<6.9$ quasars we present near-infrared spectroscopy and estimate the central black hole mass from their C$\rm IV$ and Mg$\rm II$ broad emission this http URL masses (log[M$_{\rm BH,MgII}$]$=8.58-9.14~\rm M_{\odot}$) and Eddington ratios ($\lambda_{\rm Edd,MgII}=0.74-2.2$) are consistent with other $z>6$ quasars reported in the literature. A $z = 6.3$ quasar exhibits a velocity difference of approximately $9000$ km s$^{-1}$ between the C$\rm IV$ and Mg$\rm II$ emission lines, making it one of the most extreme C$\rm IV$ outflows currently known. Additionally, the sample includes three high-ionization broad absorption line quasars. One of these quasars shows potential evidence of an extremely fast outflow feature, reaching $48000$ km s$^{-1}$.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.16846-b31b1b.svg)](https://arxiv.org/abs/2505.16846) | **Ray-tracing GR-MHD-generated Outflows from AGNs Hosting Thin Accretion Disks: An Analysis Approaching Horizon Scales**  |
|| B. Bandyopadhyay, et al. -- incl., <mark>C. Fendt</mark> |
|*Appeared on*| *2025-05-23*|
|*Comments*| *13 pages, 8 figures, 1 table, accepted for publication in A&A*|
|**Abstract**|            AGNs exhibit a wide range of black hole masses and inflow/outflow properties. It is now possible to probe regions close to the event horizons of nearby SMBHs using VLBI with earth-sized baselines, as performed by the EHT. This study explores the emission properties of accretion and outflows near the event horizon of both low-mass and high-mass SMBHs. Using resistive GR-MHD simulations, we model AGNs with thin Keplerian disks. This contrasts with widely studied models featuring thick disks, such as magnetically arrested disks (MADs) or the standard and normal evolution (SANE) scenario. Our models serve as simplified representations to study disk-jet-wind structures. These simulations are postprocessed and ray-traced, using constraints of black hole mass and observed SEDs. Thermal synchrotron emission generated near the event horizon is used to create emission maps, which are analysed by separating accretion and outflow components to determine their contributions to the total intensity. Whether the emission appears optically thick or thin at a given frequency depends on its position relative to the synchrotron SED peak. At 230 GHz, low-mass SMBHs appear optically thicker than high-mass ones, even at lower accretion rates. Doppler beaming affects the brightness of emission from outflows with changing viewing angles in low-mass systems. Eddington ratios from our models align with those inferred by the EHTC for M87 and SgrA* using thicker MAD/SANE models. Although thin disks are optically thicker, their spectral properties make high-mass systems appear optically thinner at 230 GHz; ideal for probing GR effects like photon rings. In contrast, low-mass systems remain optically thicker at these frequencies because of synchrotron self-absorption, making outflow emissions near the horizon more pronounced. However, distinguishing these features remains challenging with current EHT resolution.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.15896-b31b1b.svg)](https://arxiv.org/abs/2505.15896) | **A pulsar-helium star compact binary system formed by common envelope evolution**  |
|| Z.L.Yang, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2025-05-23*|
|*Comments*| *26+25 pages, 4+8 figures, 1+3 tables. Published on Science in the 14 May issue of Science. Authors' version*|
|**Abstract**|            A stellar common envelope occurs in a binary system when the atmosphere of an evolving star expands to encompass an orbiting companion object. Such systems are predicted to evolve rapidly, ejecting the stellar envelope and leaving the companion in a tighter orbit around a stripped star. We used radio timing to identify a pulsar, PSR J1928+1815, with a spin period of 10.55 ms in a compact binary system with an orbital period of 3.60 hours. The companion star has 1.0 to 1.6 solar masses, eclipses the pulsar for about 17% of the orbit, and is undetected at other wavelengths, so it is most likely a stripped helium star. We interpret this system as having recently undergone a common envelope phase, producing a compact binary.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.15893-b31b1b.svg)](https://arxiv.org/abs/2505.15893) | **Calibration of Binary Population Synthesis Models Using White Dwarf Binaries from APOGEE, GALEX and Gaia**  |
|| A. C. Rubio, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-05-23*|
|*Comments*| *12 pages, submitted to A&A. Comments welcome!*|
|**Abstract**|            The effectiveness and stability of mass transfer in binaries system are crucial in determining its final product. Rapid binary population synthesis (BPS) codes simplify the complex physics of mass transfer by adopting parameterized prescriptions for the stability of mass transfer, accretion efficiency in stable mass transfer, and the efficiency of common-envelope ejection. We calibrate these uncertain parameters by comparing BPS models with observational data. White dwarf and main sequence binaries are an ideal population to study binary interaction, as they can be formed through stable or unstable mass transfer, or without interaction, which affect the orbital period and masses of the present-day population. The APOGEE-GALEX-Gaia catalog provides a homogeneous sample of over 500 systems with well measured radial velocities that can be used as a comparison baseline for BPS simulations of such binaries. We compare the distribution of observed maximum radial velocity variation ($\Delta RV_{\rm max}$) and estimated masses to BPS models simulated with COSMIC, varying the mass transfer and common-envelope ejection efficiency, and the criteria for mass transfer stability at key evolutionary stages. The $\Delta RV_{\rm max}$ comparison shows clear preference for a higher fraction of stable mass transfer during the first ascent giant branch, and for highly effective envelope ejection. For the systems with WD masses, there is a slight preference for non-conservative mass transfer. In COSMIC and similar codes, the envelope ejection efficiency and the envelope binding energy are degenerate parameters. Our result of high ejection efficiency may indicate that either additional sources of energy are required to eject the envelope, or that its binding energy is lower than traditionally assumed. Future comparisons to BPS simulations can be drawn for other datasets as they become available.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.16562-b31b1b.svg)](https://arxiv.org/abs/2505.16562) | **A Machine Learning Pipeline for Hunting Hidden Axion Signals in Pulsar Dispersion Measurements**  |
|| H. Shi, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-05-23*|
|*Comments*| *9 pages, 7 figures*|
|**Abstract**|            In the axion model, electromagnetic waves interacting with axions induce frequency-dependent time delays, determined by the axion mass and decay constant. These small delays are difficult to detect, making traditional methods ineffective. To address this, we computed time delays for various parameters and found a prominent dispersion signal when the wave frequency equals half the axion mass. Based on this, we developed a machine learning-based pipeline, achieving 95\% classification accuracy and demonstrating strong detection capability in low signal-to-noise data. Applying this to PSR J1933-6211, we found no axion-induced delays within current sensitivity limits. While existing constraints are limited by atomic clock resolution in radio telescopes, future advances in optical clocks and broader bandwidths will enable more extensive searches. In particular, combining high-precision optical clocks with next-generation radio telescopes, such as the Qitai Radio Telescope, could improve decay constant constraints by four orders of magnitude for axion masses in the $10^{-6} \sim 10^{-4}$ eV range.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.15995-b31b1b.svg)](https://arxiv.org/abs/2505.15995) | **NIRCam yells at cloud: JWST MIRI imaging can directly detect exoplanets of the same temperature, mass, age, and orbital separation as Saturn and Jupiter**  |
|| R. Bowens-Rubin, et al. -- incl., <mark>E. Matthews</mark> |
|*Appeared on*| *2025-05-23*|
|*Comments*| **|
|**Abstract**|            NIRCam and MIRI coronagraphy have successfully demonstrated the ability to directly image young sub-Jupiter mass and mature gas-giant exoplanets. However, these modes struggle to reach the sensitivities needed to find the population of cold giant planets that are similar to our own Solar System's giant planets ($T_{\rm eff} = 60 - 125$ K; $a=5 - 30$ AU). For the first time, we explore the high-contrast imaging capabilities of MIRI imaging rather than coronagraphy. Using data from the JWST GO 6122: Cool Kids on the Block program which targets nearby ($<6$ pc) M-dwarfs with NIRCam coronagraphy and MIRI imaging, we demonstrate that 21$\mu$m MIRI imaging can detect planets with the same temperature, mass, age, and orbital separations as Saturn and Jupiter. For systems within 3pc, 21$\mu$m MIRI imaging reaches the sensitivity needed to detect planets colder than Saturn ($<95$ K). NIRCam coronagraphy can achieve similar results only in the unlikely case that a cold giant planet is cloud-free. Motivated by these compelling findings, we extend our analysis to evaluate the measured performance of MIRI F2100W imaging versus NIRCam F444W coronagraphy to 70 pc and conclude that MIRI imaging offers the advantage for systems within 20pc. Microlensing surveys predict an occurrence rate as high as 1 - 2 low-mass giant exoplanets per star, suggesting that JWST MIRI imaging surveys of nearby systems may be poised to uncover a substantial population. This breakthrough enables a path towards the first direct characterization of cold giant exoplanets that are analogous to the solar system giant planets.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.15923/./figures/optnir_spec_new3.png', 'tmp_2505.15923/./figures/redshift_vs_zZZy_with_histograms_v3.png', 'tmp_2505.15923/./figures/opt_total_test1_new_ldss3_portrait.png', 'tmp_2505.15923/./figures/opt_total_test2_new_ldss3_portrait.png']
copying  tmp_2505.15923/./figures/optnir_spec_new3.png to _build/html/
copying  tmp_2505.15923/./figures/redshift_vs_zZZy_with_histograms_v3.png to _build/html/
copying  tmp_2505.15923/./figures/opt_total_test1_new_ldss3_portrait.png to _build/html/
copying  tmp_2505.15923/./figures/opt_total_test2_new_ldss3_portrait.png to _build/html/
exported in  _build/html/2505.15923.md
    + _build/html/tmp_2505.15923/./figures/optnir_spec_new3.png
    + _build/html/tmp_2505.15923/./figures/redshift_vs_zZZy_with_histograms_v3.png
    + _build/html/tmp_2505.15923/./figures/opt_total_test1_new_ldss3_portrait.png
    + _build/html/tmp_2505.15923/./figures/opt_total_test2_new_ldss3_portrait.png
found figures ['tmp_2505.16846/./allsimi90M87

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\nqsos}{25}$</div>



<div id="title">

# Discovery and characterization of 25 new quasars at $4.6 < z < 6.9$ from wide-field multi-band surveys

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.15923-b31b1b.svg)](https://arxiv.org/abs/2505.15923)<mark>Appeared on: 2025-05-23</mark> -  _25 pages, 8 figures, 12 tables, Accepted for publication in A&A_

</div>
<div id="authors">

<mark>S. Belladitta</mark>, et al. -- incl., <mark>E. Bañados</mark>, <mark>Z.-L. Xie</mark>, <mark>J. Wolf</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** Luminous quasars at $z>4$ provide key insights into the early Universe.Their rarity necessitates wide-field multi-band surveys to efficiently separate them from the main astrophysical contaminants (i.e., ultracool dwarfs).To expand the sample of high- $z$ quasars, we conducted targeted selections using optical, infrared, and radio surveys, complemented by literature-based quasar candidate catalogs.In this paper, we report the discovery of $\nqsos$ new quasars at $4.6<z<6.9$ (six at $z\geq6.5$ ), with $M_{1450}$ between $-$ 25.4 and $-$ 27.0.We also present new spectra of six $z>6.5$ quasars we selected, but whose independent discovery has already been published in the literature.Three of the newly discovered quasars are strong radio emitters (L $_{1.4 \rm GHz}=0.09-1.0\times$ 10 $^{34}$ erg s $^{-1}$ Hz $^{-1}$ ).Among them, one source at $z=4.71$ exhibits typical blazar-like properties, including a flat radio spectrum, radio-loudness $\sim$ 1000, and multi-frequency variability.It is also detected by SRG/eROSITA X-ray telescope (f $_{\rm 0.2-2.3keV} \sim 1.3\times10^{-13}$ erg s $^{-1}$ cm $^{-2}$ ).In addition, for seven $6.3<z<6.9$ quasars we present near-infrared spectroscopy and estimate the central black hole mass from their C $\rm IV$ and Mg $\rm II$ broad emission lines.Their masses (log [ M $_{\rm BH,MgII}$ ] $=8.58-9.14 \rm M_{\odot}$ ) and Eddington ratios ( $\lambda_{\rm Edd,MgII}=0.74-2.2$ ) are consistent with other _z_ $>$ 6 quasars reported in the literature.A $z = 6.3$ quasar exhibits a velocity difference of approximately $9000$ km s $^{-1}$ between the C $\rm IV$ and Mg $\rm II$ emission lines, making it one of the most extreme C $\rm IV$ outflows currently known.Additionally, the sample includes three high-ionization broad absorption line quasars. One of these quasars shows potential evidence of an extremely fast outflow feature, reaching $48 000$ km s $^{-1}$ .

</div>

<div id="div_fig1">

<img src="tmp_2505.15923/./figures/optnir_spec_new3.png" alt="Fig6" width="100%"/>

**Figure 6. -** \small NIR follow-up spectra of a sub-sample of quasars reported in this paper (sorted by increasing redshift). We show the NIR spectra of the newly discovered quasars PSO J041$+$06, PSO J335$-$15, PSO J067$-$14, PSO J217$+$04, PSO J037$-$08 and PSO J289$+$50 and the newly published NIR spectrum for the already known quasar PSO J162$-$01.
    The noise spectra are reported in gray.
    Shaded light gray areas highlight regions strongly affected by telluric absorption.
    On the top the location of the main emission lines according to the Mg$\rm II$ redshift are also shown: Lyman-$\alpha+$N$\rm V$(dashed), C$\rm IV$(dotted) and Mg$\rm II$(dashed-dotted).  (*fig:nirspectra*)

</div>
<div id="div_fig2">

<img src="tmp_2505.15923/./figures/redshift_vs_zZZy_with_histograms_v3.png" alt="Fig1" width="100%"/>

**Figure 1. -** \small Redshift vs.  $z_{\rm P1} - z_{\rm DE}$(bottom) and $z_{\rm DE}-y_{\rm P1}$(top) colors. The blue, black, and red solid lines represent the color tracks of composite quasar spectra from [Bañados, Venemans and Decarli (2016)](), illustrating weak, average, and strong Ly$\alpha$ emission lines, respectively.
    The dotted lines indicate the color cuts for the [DELS+PS1] selection, as described in Section \ref{selectionbanados23}. The red circles represent quasars that meet the selection criteria. Sources marked with a yellow cross are newly discovered quasars from this study, while those with a white `x' are known quasars for which we present new spectroscopy data. Lower limits correspond to sources undetected in $z_{\rm P1}$, for which we use their $3\sigma$ limiting magnitude. On the right, we show the histograms of the same colors for the L and T dwarfs compiled in [Bañados, Venemans and Decarli (2016)](). (*fig:z66sel*)

</div>
<div id="div_fig3">

<img src="tmp_2505.15923/./figures/opt_total_test1_new_ldss3_portrait.png" alt="Fig5.1" width="50%"/><img src="tmp_2505.15923/./figures/opt_total_test2_new_ldss3_portrait.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** \small Newly discovered spectra (25 sources, $*$ marked radio-loud objects) and new spectra publication (6 objects, marked with a $\star$) for the quasars reported in this paper. Only the part of the spectrum that covers the Lyman-$\alpha$ break is shown here. The noise spectrum is reported in gray. Sorted by increasing redshift. (*fig:spectra_lyman*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.15923"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Ray-tracing GR-MHD-generated outflows from AGNs hosting thin accretion disks: An analysis approaching horizon scales

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.16846-b31b1b.svg)](https://arxiv.org/abs/2505.16846)<mark>Appeared on: 2025-05-23</mark> -  _13 pages, 8 figures, 1 table, accepted for publication in A&A_

</div>
<div id="authors">

B. Bandyopadhyay, et al. -- incl., <mark>C. Fendt</mark>

</div>
<div id="abstract">

**Abstract:** Active galactic nuclei (AGNs) exhibit a wide range of black hole masses and inflow/outflow properties. It is now possible to probe regions close to the event horizons of nearby supermassive black holes (SMBHs) using very long baseline interferometry (VLBI) with earth-sized baselines, as performed by the Event Horizon Telescope (EHT). This study explores the emission properties of accretion and outflows near the event horizon of both low-mass and high-mass SMBHs. Using resistive general relativistic magnetohydrodynamic (GR-MHD) simulations, we model AGNs with thin Keplerian disks. This contrasts with widely studied models featuring thick disks, such as magnetically arrested disks (MADs) or the standard and normal evolution (SANE) scenario. Our GR-MHD models serve as simplified representations to study disk–jet–wind structures. These simulations are postprocessed and ray-traced, using constraints of black hole mass and observed spectral energy distributions (SEDs). Thermal synchrotron emission generated near the event horizon is used to create emission maps, which are analysed by separating accretion and outflow components to determine their contributions to the total intensity. Whether the emission appears optically thick or thin at a given frequency depends on its position relative to the synchrotron SED peak. At 230 GHz, low-mass SMBHs appear optically thicker than high-mass ones, even at lower accretion rates. Doppler beaming affects the brightness of emission from outflows with changing viewing angles in low-mass systems. Eddington ratios from our models align with those inferred by the EHTC for M87 and SgrA* using thicker MAD/SANE models. Although thin disks are optically thicker, their spectral properties make high-mass systems appear optically thinner at 230 GHz—ideal for probing GR effects like photon rings. In contrast, low-mass systems remain optically thicker at these frequencies because of synchrotron self-absorption, making outflow emissions near the horizon more pronounced. However, distinguishing these features remains challenging with current EHT resolution.

</div>

<div id="div_fig1">

<img src="tmp_2505.16846/./allsimi90M87SgrAfallfthR2712Rh80rev.png" alt="Fig6.1" width="50%"/><img src="tmp_2505.16846/./allsimlowiM87SgrAf230fthR12Rh80rev.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** _ Left:_ Normalized emission maps (25 $R_g$ across) for an edge-on view for all our simulations applied for HIGH-mass (upper 3 rows) and LOW-mass (lower 3 rows) systems
   at 86 GHz, 230 GHz and 345 GHz obtained with the Eddington ratio detailed in Table \ref{tab:para_dynamics}. _ Right:_ Normalized 230 GHz emission maps (25 $R_g$ across) as above but for inclination angles of $0^{\circ}$, $17^{\circ}$, and $163^{\circ}$. (*fig:allsimalli*)

</div>
<div id="div_fig2">

<img src="tmp_2505.16846/./specSim20M87SgrAcomrev.png" alt="Fig5.1" width="16%"/><img src="tmp_2505.16846/./specSim21M87SgrAcomrev.png" alt="Fig5.2" width="16%"/><img src="tmp_2505.16846/./specSim22M87SgrAcomrev.png" alt="Fig5.3" width="16%"/><img src="tmp_2505.16846/./specSim23M87SgrAcomrev.png" alt="Fig5.4" width="16%"/><img src="tmp_2505.16846/./specSim24M87SgrAcomrev.png" alt="Fig5.5" width="16%"/><img src="tmp_2505.16846/./specSim26M87SgrAcomrev.png" alt="Fig5.6" width="16%"/>

**Figure 5. -** SEDs applying Eddington ratios from Table \ref{tab:para_dynamics} for a HIGH-mass (blue curves) and LOW-mass (orange curves) system for SIM20, SIM21, SIM22, SIM23, SIM24, and SIM26 respectively. The three vertical lines correspond to 86 GHz, 230 GHz, and 345 GHz, respectively. The figures also mark the optically thin and optically thick regions of the thermal synchrotron, which lie on the right and left sides of the thermal synchrotron peak marked by the red diamond. (*fig:specdiffM87SgrA*)

</div>
<div id="div_fig3">

<img src="tmp_2505.16846/./AllSimi17M87SgrAdiskoutflowf230fthR12Rh80rev.png" alt="Fig2" width="100%"/>

**Figure 2. -** Normalized 230 GHz emission maps (25 $R_g$ across) for all the models with the total emissions, only the disk emissions and the emission from the "outflow" region for HIGH-mass (left 3 columns) and LOW-mass (right 3 columns) systems for an inclination angle of $17^{\circ}$. (*fig:prothdiskoutflowalli1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.16846"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

498  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
